<a href="https://colab.research.google.com/github/tanaerao/bench-calls/blob/main/negotiators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

def generate_negotiations(num_of_negotiations):
    buyer_lines = []
    seller_lines = []

    for _ in range(num_of_negotiations):
        v = random.randint(1, 10)
        c = random.randint(1, 8)
        offer = random.randint(1, 10)

        # Buyer accepts only if their valuation is lower or equal to the offer
        accept_reject = "ACCEPT" if v >= offer else "REJECT"

        buyer_reward = v - offer if accept_reject == "ACCEPT" else 0
        seller_reward = offer - c if accept_reject == "ACCEPT" else 0

        buyer_negotiation = (
            "START_NEGOTIATION_BUYER\n"
            f"v: {v}\n"
            f"seller_offer: {offer}\n"
            f"buyer_accept_reject: {accept_reject}\n"
            f"buyer_reward: {buyer_reward}\n"
            "END_NEGOTIATION_BUYER\n"
        )
        seller_negotiation = (
            "START_NEGOTIATION_SELLER\n"
            f"c: {c}\n"
            f"seller_offer: {offer}\n"
            f"buyer_accept_reject: {accept_reject}\n"
            f"seller_reward: {seller_reward}\n"
            "END_NEGOTIATION_SELLER\n"
        )

        buyer_lines.append(buyer_negotiation)
        seller_lines.append(seller_negotiation)

    with open('buyer_negotiations.txt', 'w') as buyer_file:
        buyer_file.writelines(buyer_lines)

    with open('seller_negotiations.txt', 'w') as seller_file:
        seller_file.writelines(seller_lines)

# Example usage
num_of_negotiations = 10000  # Adjust this value as needed
generate_negotiations(num_of_negotiations)



In [ ]:
# Read buyer negotiations from file
with open('buyer_negotiations.txt', 'r') as buyer_file:
    text = buyer_file.read()


print("Buyer Negotiations:\n", text[:100]) # Print the first 100 characters of buyer_text

Buyer Negotiations:
 START_NEGOTIATION_BUYER
v: 2
seller_offer: 9
buyer_accept_reject: REJECT
buyer_reward: 0
END_NEGOTIA


In [ ]:
# tokenizer

import re

# Define the mappings
mappings = {
    "\n": 0,
    " ": 1,
    "START_NEGOTIATION_BUYER": 2,
    "v:": 3,
    "seller_offer:": 4,
    "buyer_accept_reject:": 5,
    "ACCEPT": 6,
    "REJECT": 7,
    "buyer_reward:": 8,
    "END_NEGOTIATION_BUYER": 9,
}

# Extend mappings for integers from -10 to 10
for i in range(-10, 11):
    mappings[str(i)] = 10 + (i + 10)

# Reverse the mappings for decoding
reverse_mappings = {v: k for k, v in mappings.items()}

# Encode function
def encode(buyer_text):
    encoded_text = []
    for line in buyer_text.splitlines():
        for word in re.split(r'(\s+)', line):  # Split by whitespace while keeping them
            if word in mappings:
                encoded_text.append(mappings[word])
            elif word.lstrip('-').isdigit() and int(word) in range(-10, 11):
                encoded_text.append(mappings[word])
        encoded_text.append(mappings["\n"])  # Add newline character after each line
    return encoded_text

# Decode function
def decode(encoded_text):
    decoded_text = ""
    for code in encoded_text:
        if code in reverse_mappings:
            decoded_text += reverse_mappings[code]
        else:
            decoded_text += str(code - 10 - 10)  # For integers in the range -10 to 10
    return decoded_text

# Example usage
with open('buyer_negotiations.txt', 'r') as buyer_file:
    buyer_text = buyer_file.read()

encoded_buyer_text = encode(buyer_text)
print("Encoded Buyer Text:\n", encoded_buyer_text[:100])

decoded_buyer_text = decode(encoded_buyer_text)
print("\nDecoded Buyer Text:\n", decoded_buyer_text[:100])


Encoded Buyer Text:
 [2, 0, 3, 1, 22, 0, 4, 1, 29, 0, 5, 1, 7, 0, 8, 1, 20, 0, 9, 0, 2, 0, 3, 1, 24, 0, 4, 1, 28, 0, 5, 1, 7, 0, 8, 1, 20, 0, 9, 0, 2, 0, 3, 1, 25, 0, 4, 1, 24, 0, 5, 1, 6, 0, 8, 1, 21, 0, 9, 0, 2, 0, 3, 1, 22, 0, 4, 1, 22, 0, 5, 1, 6, 0, 8, 1, 20, 0, 9, 0, 2, 0, 3, 1, 27, 0, 4, 1, 26, 0, 5, 1, 6, 0, 8, 1, 21, 0, 9, 0]

Decoded Buyer Text:
 START_NEGOTIATION_BUYER
v: 2
seller_offer: 9
buyer_accept_reject: REJECT
buyer_reward: 0
END_NEGOTIA


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(buyer_text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([200000]) torch.int64
tensor([ 2,  0,  3,  1, 22,  0,  4,  1, 29,  0,  5,  1,  7,  0,  8,  1, 20,  0,
         9,  0,  2,  0,  3,  1, 24,  0,  4,  1, 28,  0,  5,  1,  7,  0,  8,  1,
        20,  0,  9,  0,  2,  0,  3,  1, 25,  0,  4,  1, 24,  0,  5,  1,  6,  0,
         8,  1, 21,  0,  9,  0,  2,  0,  3,  1, 22,  0,  4,  1, 22,  0,  5,  1,
         6,  0,  8,  1, 20,  0,  9,  0,  2,  0,  3,  1, 27,  0,  4,  1, 26,  0,
         5,  1,  6,  0,  8,  1, 21,  0,  9,  0,  2,  0,  3,  1, 24,  0,  4,  1,
        27,  0,  5,  1,  7,  0,  8,  1, 20,  0,  9,  0,  2,  0,  3,  1, 30,  0,
         4,  1, 23,  0,  5,  1,  6,  0,  8,  1, 27,  0,  9,  0,  2,  0,  3,  1,
        21,  0,  4,  1, 21,  0,  5,  1,  6,  0,  8,  1, 20,  0,  9,  0,  2,  0,
         3,  1, 25,  0,  4,  1, 23,  0,  5,  1,  6,  0,  8,  1, 22,  0,  9,  0,
         2,  0,  3,  1, 24,  0,  4,  1, 26,  0,  5,  1,  7,  0,  8,  1, 20,  0,
         9,  0,  2,  0,  3,  1, 22,  0,  4,  1, 30,  0,  5,  1,  7,  0,  8,  1,
       

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
vocab_size = len(mappings)
print(vocab_size)

31


BREAK

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 19 # what is the maximum context length for predictions? <-- changed by me from 32 due to negotiation length in tokens
max_iters = 3500
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.204511 M parameters
step 0: train loss 3.6110, val loss 3.6154
step 100: train loss 0.4964, val loss 0.4922
step 200: train loss 0.3971, val loss 0.4002
step 300: train loss 0.3921, val loss 0.3951
step 400: train loss 0.3841, val loss 0.3872
step 500: train loss 0.3825, val loss 0.3820
step 600: train loss 0.3752, val loss 0.3788
step 700: train loss 0.3689, val loss 0.3729
step 800: train loss 0.3641, val loss 0.3684
step 900: train loss 0.3687, val loss 0.3692
step 1000: train loss 0.3650, val loss 0.3720
step 1100: train loss 0.3635, val loss 0.3671
step 1200: train loss 0.3602, val loss 0.3626
step 1300: train loss 0.3608, val loss 0.3650
step 1400: train loss 0.3561, val loss 0.3563
step 1500: train loss 0.3526, val loss 0.3555
step 1600: train loss 0.3534, val loss 0.3570
step 1700: train loss 0.3503, val loss 0.3536
step 1800: train loss 0.3488, val loss 0.3522
step 1900: train loss 0.3514, val loss 0.3548
step 2000: train loss 0.3478, val loss 0.3528
step 2100: train loss 0.

evals

In [ ]:
def calculate_correct_behavior_proportion_from_string(negotiation_text):
    lines = negotiation_text.splitlines()

    in_negotiation = False
    total_negotiations = 0
    correct_behavior_count = 0

    for line in lines:
        line = line.strip()
        if line == "START_NEGOTIATION_BUYER":
            in_negotiation = True
            v = None
            offer = None
            accept_reject = None
        elif line == "END_NEGOTIATION_BUYER":
            if in_negotiation and v is not None and offer is not None and accept_reject is not None:
                total_negotiations += 1
                if (v >= offer and accept_reject == "ACCEPT") or (v < offer and accept_reject == "REJECT"):
                    correct_behavior_count += 1
            in_negotiation = False
        elif in_negotiation:
            try:
                if line.startswith("v:"):
                    v = int(line.split(":")[1].strip())
                elif line.startswith("seller_offer:"):
                    offer = int(line.split(":")[1].strip())
                elif line.startswith("buyer_accept_reject:"):
                    accept_reject = line.split(":")[1].strip()
            except ValueError as e:
                print(f"Error parsing line: {line}. Error: {e}")
                # Skip this negotiation if there's a parsing error
                in_negotiation = False

    if total_negotiations == 0:
        return 0  # To handle division by zero if there are no complete negotiations

    proportion = correct_behavior_count / total_negotiations
    return proportion

# Example usage
negotiation_text = """
START_NEGOTIATION_BUYER
v: 5
seller_offer: 7
buyer_accept_reject: REJECT
buyer_reward: 0
END_NEGOTIATION_BUYER
START_NEGOTIATION_BUYER
v: 8
seller_offer: 6
buyer_accept_reject: ACCEPT
buyer_reward: 2
END_NEGOTIATION_BUYER
"""  # Adjust this string as needed

proportion = calculate_correct_behavior_proportion_from_string(negotiation_text)
print(f"Proportion of correct behavior: {proportion}")


Proportion of correct behavior: 1.0


In [ ]:
calculate_correct_behavior_proportion_from_string(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

1.0